In [ ]:
!pip install mlxtend

In [315]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import glob
from sklearn.feature_selection import VarianceThreshold
import gc
import categorizer

In [316]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [317]:
df = pd.read_csv('data/confirmed_members/post_cov_w_days.csv')

In [328]:
t = df[(df['cov_19']=='Confirmed')].groupby('clm_patient_id')['cl_fst_serv_dt'].min().reset_index()

In [331]:
df = pd.merge(df, t, on='clm_patient_id')

In [332]:
df.head()

,cl_fst_serv_dt_x,cl_proc_cd,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,cov_19,invn_ctl_nbr,Unnamed: 0,clm_patient_id,days,test,cl_fst_serv_dt_y
0,2020-04-10,99214,R05,Z03818,I10,E119,NaN,Non-COVID,CA74024626,7754,89010861901F19720915,1,NaN,2020-04-17
1,2020-04-17,99213,I10,H01139,U071,J189,NaN,Confirmed,CA89975181,2024705,89010861901F19720915,8,NaN,2020-04-17
2,2020-04-13,99291,J9601,U071,NaN,NaN,NaN,Confirmed,3439573209,400,29585450118F19361108,8,NaN,2020-04-07
3,2020-04-09,99291,J9601,U071,NaN,NaN,NaN,Confirmed,3439573212,4871,29585450118F19361108,4,NaN,2020-04-07
4,2020-04-10,99291,J9601,U071,NaN,NaN,NaN,Confirmed,3439573213,11777,29585450118F19361108,5,NaN,2020-04-07


In [299]:
df_members = df.groupby(['clm_patient_id', 'cov_19'])['invn_ctl_nbr'].count().reset_index()

In [300]:
df_members.head()

,clm_patient_id,cov_19,invn_ctl_nbr
0,00000020401M19470211,Confirmed,6
1,00000020401M19470211,Non-COVID,3
2,00000027018M19600305,Confirmed,4
3,00000027018M19600305,Non-COVID,12
4,00000033318M19481113,Confirmed,1


In [273]:
len(df['clm_patient_id'].unique())

14559

In [274]:
df_all = df_members[(df_members['cov_19']=='Confirmed') & (df_members['invn_ctl_nbr']>1)]

In [275]:
len(df_all['clm_patient_id'].unique())

8043

In [276]:
df_all.head()

,clm_patient_id,cov_19,invn_ctl_nbr
0,00000020401M19470211,Confirmed,6
2,00000027018M19600305,Confirmed,4
7,00000042001F19761022,Confirmed,2
10,00000078218M19611222,Confirmed,7
20,00000272218M19510830,Confirmed,6


In [277]:
df_all['invn_ctl_nbr'].mean()

7.862489120974761

In [278]:
df_all['invn_ctl_nbr'].value_counts()

2     1803
3     1218
4      885
5      608
6      507
      ... 
74       1
78       1
82       1
63       1
69       1
Name: invn_ctl_nbr, Length: 70, dtype: int64

In [279]:
#df_v.to_csv('data/confirmed_members/more_u071_claims.csv', index=False)

In [280]:
df_all['invn_ctl_nbr'].sum()/len(df['clm_patient_id'].unique())

4.343567552716533

In [311]:
df_t = pd.read_csv('data/confirmed_members/post_cov_w_days.csv')

In [312]:
df_t.head()

,Unnamed: 0,cl_fst_serv_dt_x,cl_proc_cd,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,cov_19,invn_ctl_nbr,Unnamed: 0.1,cl_fst_serv_dt_y,clm_patient_id,days,clm_total_charge
0,1,2020-04-10,99214,R05,Z03818,I10,E119,NaN,Non-COVID,CA74024626,7754.0,2020-04-10,89010861901F19720915,1,274.00
1,2,2020-04-17,99213,I10,H01139,U071,J189,NaN,Confirmed,CA89975181,2024705.0,2020-04-17,89010861901F19720915,8,188.00
2,3,2020-04-17,99222,Z515,U071,F0390,NaN,NaN,Confirmed,CA74007325,115.0,2020-04-17,90249432418M19321201,3,485.00
3,4,2020-04-17,1123F,Z515,U071,NaN,NaN,NaN,Confirmed,CA74007326,697.0,2020-04-17,90249432418M19321201,3,0.01
4,5,2020-04-20,99231,Z515,U071,F0390,NaN,NaN,Confirmed,CA74007327,25403.0,2020-04-20,90249432418M19321201,6,139.00


In [313]:
df_t['days'].value_counts()

1     25204
2     20472
3     18206
4     16164
5     14896
6     13285
7     12446
8     10257
9      8768
10     7771
11     7088
12     6696
13     6145
14     5769
15     4753
16     4192
17     3435
18     3177
19     3095
20     2887
21     2619
22     2097
23     1693
24     1324
25     1173
26     1142
27      967
28      777
29      546
30      429
Name: days, dtype: int64

In [283]:
df_joined = pd.merge(left=df_all, right=df_t, on='clm_patient_id')

In [284]:
df_joined

,clm_patient_id,cov_19_x,invn_ctl_nbr_x,Unnamed: 0,cl_fst_serv_dt_x,cl_proc_cd,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,cov_19_y,invn_ctl_nbr_y,Unnamed: 0.1,cl_fst_serv_dt_y,days,clm_total_charge
0,00000020401M19470211,Confirmed,6,114258,2020-04-12,99291,E1110,E119,U071,NaN,NaN,Confirmed,CA56136226,1281696.0,2020-04-12,4,1415.0
1,00000020401M19470211,Confirmed,6,114259,2020-04-10,99291,E1110,E119,U071,NaN,NaN,Confirmed,CA56136401,1452218.0,2020-04-10,2,2830.0
2,00000020401M19470211,Confirmed,6,114260,2020-04-09,93010,R9431,I2119,I2129,I440,NaN,Non-COVID,CA56136402,1468830.0,2020-04-09,1,44.0
3,00000020401M19470211,Confirmed,6,114261,2020-04-09,99291,U071,E1110,J1289,NaN,NaN,Confirmed,CA56136717,1606609.0,2020-04-09,1,1415.0
4,00000020401M19470211,Confirmed,6,114262,2020-04-14,99239,E1110,E119,U071,J1289,NaN,Confirmed,CA59602689,903005.0,2020-04-14,6,1075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110838,YLS89093268418M19730420,Confirmed,5,273712,2020-04-06,71045,Z4682,J1289,B9729,Z4659,NaN,Probable,CA74364524,176872.0,2020-04-06,4,96.0
110839,YLS89093268418M19730420,Confirmed,5,273713,2020-04-09,71045,U071,J1289,Z4659,NaN,NaN,Confirmed,CA74364515,198355.0,2020-04-09,7,64.0
110840,YLS89093268418M19730420,Confirmed,5,273714,2020-04-03,71045,U071,J1289,NaN,NaN,NaN,Confirmed,CA74364523,199201.0,2020-04-03,1,32.0
110841,ZG72417E18F19610507,Confirmed,2,157413,2020-04-08,99233,U071,J189,J45909,L02811,NaN,Confirmed,CA70327519,626785.0,2020-04-08,1,306.0


In [333]:
def return_proc(x):
    if len(x) > 5:
        print(x)
        x = x[:5]
    elif len(x) < 5:
        return 'None'
    else:
        pass

    if x[0] == 'G':
        return 'Temporary Procedures & Professional Services (G)'
    elif x[4] == 'F':
        if x[0] == '1':
            return 'Patient History'
        elif x[0] == '4':
            return 'Therapeutic, Preventative or Other Interventions'
        elif x[0] == '3':
            return 'Diagnostic/Screening Processes or Results'
        elif x[0] == '2':
            return 'Physical Examination'
    elif x[0] == 'Q':
        return 'Temporary Code (Q)'
    elif str(x)[:2] == '94':
        return 'Pulmonary Diagnostic Testing and Therapies'
    elif str(x) in ['99080', '99000', '99183']:
        return 'Miscellaneous Medicine Services'
    elif str(x) == '99497':
        return 'Advanced Care Planning'
    elif str(x) == '92526':
        return 'Special Otorhinolaryngologic Services and Procedures'
    elif str(x)[0] == 'V':
        return 'Vision, Hearing and Speech-Language Pathology Services'
    elif str(x)[0] == 'S':
        return 'Temporary Code - Commercial Payers'
    elif str(x) in ['92557', '92567']:
        return 'Audiology Function Tests'
    elif str(x) == '95910':
        return 'Nerve Conduction Studies'
    elif str(x) == '95886':
        return 'Needle electromyography'
    elif str(x)[0] == 'B':
        return 'Enteral and Parenteral Therapy'
    elif str(x) == '90999':
        return 'Other Dialysis Procedures'
    elif str(x)[0] == 'C':
        return 'Temporary Codes for Use with Outpatient Prospective Payment System (C)'
    elif str(x) == '92507':
        return 'Treatment of speech, language, voice, communication, and/or auditory processing disorder'
    elif str(x)[0] == 'A':
        return 'Transportation, Medical & Surgical Supplies, Miscellaneous & Experimental'
    elif str(x) in ['95923', '95924']:
        return 'Testing of autonomic nervous system function'
    elif str(x) == '99152':
        return 'Moderate (Conscious) Sedation'
    elif str(x) in ['96138', '96139', '96136']:
        return 'Test Administration and Scoring'
    elif str(x) in ['96132', '96133', '96130']:
        return 'Testing Evaluation Services'
    elif str(x)[-1] =='T':
        return 'New Technology'
    elif str(x) in ['92943', '92928', '92929', '92941']:
        return 'Coronary Therapeutic Services and Procedures'
    elif str(x) in ['93458', '93451', '93454', '93455', '93459']:
        return 'Cardiac Catheterization Procedures'
    elif str(x) in ['96127']:
        return 'Developmental/Behavioral Screening and Testing'
    elif str(x) in ['90791']:
        return 'Psychiatric Diagnostic Procedures'
    elif str(x)[-1] =='F':
        return 'Data Collection for Performance Measures'
    elif str(x) in ['96160']:
        return 'Administration of health risk assessment instrument'
    elif str(x) in ['90756']:
        return 'Influenza virus vaccine, quadrivalent'
    elif str(x)[-1] =='U':
        return 'Proprietary Lab Analysis'
    elif str(x) in ['10005', '10006']:
        return 'Fine Needle Aspiration Biopsy'
    elif str(x) in ['99490']:
        return 'Chronic Care Management Services'
    else:
        pass

    b = categorizer.outpatient_categorizer_by_hcpc(str(x))

    if (b == 'Other Outpatient Services') | (b=='BADHCPCCODE') | (b=='NOTFOUND'):
        z = categorizer.carrier_categorizer_by_hcpc(str(x))
        if (z=='BADHCPCCODE') | (z=='NOTFOUND'):
            return(x)
        else:
            return z
    else:
        return b

In [334]:
df_t = pd.read_csv('data/confirmed_members/post_cov_w_days_old.csv')

/usr/local/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [335]:
df_t['proc_desc'] = df_t['cl_proc_cd_x'].astype(str).apply(lambda x: return_proc(x))

In [338]:
df_t['proc_desc'].value_counts()

Inpatient Visits                            83444
Radiology Services: Diagnostic Radiology    41320
Office Visits                               31148
Emergency Room                              30169
Cardiovascular                              13146
                                            ...  
3IB21                                           1
Anesthesia: Other Procedures                    1
3JC21                                           1
59414                                           1
Testing Evaluation Services                     1
Name: proc_desc, Length: 208, dtype: int64

In [341]:
df_t.to_csv('data/confirmed_members/post_cov_w_days_old.csv', index=False)

In [339]:
df_t[df_t['days']==0]

,cl_fst_serv_dt,cl_proc_cd_x,clm_hc_diag_code1_x,clm_hc_diag_code2_x,clm_hc_diag_code3_x,clm_hc_diag_code4_x,clm_hc_diag_code5_x,cov_19_x,invn_ctl_nbr,Unnamed: 0_x,...,clm_patient_zip,clm_total_charge,cov_19_y,invn_ctl_nbr_x,Unnamed: 0_y,cl_fst_serv_dt_y,clm_patient_id,invn_ctl_nbr_y,days,proc_desc


In [305]:
df_joined['proc_desc'] = df_joined['cl_proc_cd'].astype(str).apply(lambda x: return_proc(x))

In [306]:
df_joined['proc_desc_desc'].value_counts()

Inpatient Visits                            47998
Radiology Services: Diagnostic Radiology    18855
Emergency Room                              15098
Office Visits                                5698
Cardiovascular                               5569
                                            ...  
59515                                           1
1BHMS                                           1
96158                                           1
59400                                           1
Nerve Conduction Studies                        1
Name: proc_desc, Length: 151, dtype: int64

In [307]:
df_joined.to_csv('data/confirmed_members/count_u071_joined.csv', index=False)

In [285]:
df_joined['invn_ctl_nbr_x'].value_counts()

2     8417
3     7891
4     7277
5     6049
6     5851
      ... 
61      74
65      72
56      70
68      69
53      51
Name: invn_ctl_nbr_x, Length: 70, dtype: int64

In [291]:
df_v = df_joined.groupby('invn_ctl_nbr_x')['clm_total_charge'].sum().reset_index()

In [292]:
df_v

,invn_ctl_nbr_x,clm_total_charge
0,2,4481691.31
1,3,4055866.77
2,4,3715991.98
3,5,2813825.91
4,6,2803649.39
...,...,...
65,74,64996.00
66,76,30367.23
67,78,46823.50
68,79,51287.00


In [293]:
df_v.to_csv('data/confirmed_members/multi_u071_cost_sum.csv',index=False)

In [241]:
df_all.to_csv('data/confirmed_members/multi_u071_members.csv', index=False)

In [203]:
drops = []

for i in df.columns:
    if i[-1] == 'y':
        drops.append(i)

In [204]:
df.drop(drops, inplace=True, axis=1)

In [205]:
#df.drop('cl_fst_serv_dt_x', inplace=True, axis=1)

In [206]:
cls = []

for i in df.columns:
    if i[-1] == 'x':
        cls.append(i[:-2])
    else:
        cls.append(i)

In [207]:
print(cls)

['Unnamed: 0', 'cl_fst_serv_dt', 'cl_proc_cd', 'clm_hc_diag_code1', 'clm_hc_diag_code2', 'clm_hc_diag_code3', 'clm_hc_diag_code4', 'clm_hc_diag_code5', 'cov_19', 'invn_ctl_nbr', 'Unnamed: 0.1', 'clm_patient_id', 'days', 'clm_total_charge']


In [208]:
cols = ['clm_hc_diag_code1', 'clm_hc_diag_code2', 'clm_hc_diag_code3', 'clm_hc_diag_code4', 'clm_hc_diag_code5']

In [211]:
df.columns = cls

In [212]:
#for i in cols:
#    v = df.groupby([i])[i].agg(
#                    {"code_count": len}).sort_values(
#                    "code_count", ascending=False).head(100).reset_index()
#    df.loc[~df[i].isin(v[i].values), i] = "Other"

In [213]:
df.head()

,Unnamed: 0,cl_fst_serv_dt,cl_proc_cd,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,cov_19,invn_ctl_nbr,Unnamed: 0.1,clm_patient_id,days,clm_total_charge
0,1,2020-04-10,99214,R05,Z03818,I10,E119,NaN,Non-COVID,CA74024626,7754.0,89010861901F19720915,1,274.00
1,2,2020-04-17,99213,I10,H01139,U071,J189,NaN,Confirmed,CA89975181,2024705.0,89010861901F19720915,8,188.00
2,3,2020-04-17,99222,Z515,U071,F0390,NaN,NaN,Confirmed,CA74007325,115.0,90249432418M19321201,3,485.00
3,4,2020-04-17,1123F,Z515,U071,NaN,NaN,NaN,Confirmed,CA74007326,697.0,90249432418M19321201,3,0.01
4,5,2020-04-20,99231,Z515,U071,F0390,NaN,NaN,Confirmed,CA74007327,25403.0,90249432418M19321201,6,139.00


In [214]:
df_confirmed = df[(df['cov_19']=='Confirmed') & (df['cl_fst_serv_dt'] <= '2020-04-10')]

In [215]:
len(df_confirmed)

26715

In [216]:
mems = df_confirmed['clm_patient_id'].unique()

In [217]:
mems = mems.tolist()

In [218]:
len(mems)

7522

In [118]:
df_m = df[df['clm_patient_id'].isin(mems)]

In [119]:
df_m.head()

,cl_fst_serv_dt,cl_proc_cd,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,cov_19,invn_ctl_nbr,Unnamed: 0,clm_patient_id,days
0,2020-04-09,99214,I10,E785,E119,R110,U071,Confirmed,CA74024625,49,89010861901F19720915,0
1,2020-04-10,99214,R05,Z03818,I10,E119,NaN,Non-COVID,CA74024626,7754,89010861901F19720915,1
2,2020-04-17,99213,I10,H01139,U071,J189,NaN,Confirmed,CA89975181,2024705,89010861901F19720915,8
14,2020-04-13,99291,J9601,U071,NaN,NaN,NaN,Confirmed,3439573209,400,29585450118F19361108,8
15,2020-04-09,99291,J9601,U071,NaN,NaN,NaN,Confirmed,3439573212,4871,29585450118F19361108,4


In [120]:
df_m['days'].value_counts()

0     42152
1     13604
2     11129
3     10087
4      9223
5      8935
6      7942
7      7460
8      6199
9      5376
10     4890
12     4832
11     4702
13     4548
14     4268
15     3642
16     3216
19     2808
17     2796
18     2750
20     2683
21     2501
22     2034
23     1672
24     1327
25     1173
26     1148
27      967
28      777
29      546
30      429
31      204
32      185
33      157
34       39
35        1
Name: days, dtype: int64

In [121]:
df_m = df_m[df_m['days']>0]

In [122]:
df_m['days'].value_counts()

1     13604
2     11129
3     10087
4      9223
5      8935
6      7942
7      7460
8      6199
9      5376
10     4890
12     4832
11     4702
13     4548
14     4268
15     3642
16     3216
19     2808
17     2796
18     2750
20     2683
21     2501
22     2034
23     1672
24     1327
25     1173
26     1148
27      967
28      777
29      546
30      429
31      204
32      185
33      157
34       39
35        1
Name: days, dtype: int64

In [123]:
df3 = df_m[cols].melt(var_name='columns', value_name='DX').reset_index()

In [125]:
df3['DX'].value_counts()

U071       65277
J9601      25901
J1289      15931
J189       12959
N179       12217
           ...  
M50222         1
E0836          1
S32020A        1
L97212         1
H0520          1
Name: DX, Length: 4971, dtype: int64

In [126]:
df_out = df3['DX'].value_counts().reset_index()

In [128]:
df_out.to_csv('data/confirmed_members/postu071_single_dx.csv',index=False)

In [314]:
df_m['days'].value_counts()

1     13604
2     11129
3     10087
4      9223
5      8935
6      7942
7      7460
8      6199
9      5376
10     4890
12     4832
11     4702
13     4548
14     4268
15     3642
16     3216
19     2808
17     2796
18     2750
20     2683
21     2501
22     2034
23     1672
24     1327
25     1173
26     1148
27      967
28      777
29      546
30      429
31      204
32      185
33      157
34       39
35        1
Name: days, dtype: int64

In [42]:
len(df_m['clm_patient_id'].unique())

14559

In [43]:
14559/20372

0.7146573728647163

In [44]:
c = df_m[df_m['cov_19']=='Confirmed']['clm_patient_id'].unique().tolist()

In [45]:
nc = df_m[df_m['cov_19']!='Confirmed']['clm_patient_id'].unique().tolist()

In [46]:
len(set(nc) & set(c)) + len(set(nc)-set(c)) + len(set(c) - set(nc))

14559

In [47]:
#All members with U071 between April 1 and April 10
len(df_m['clm_patient_id'].unique())

14559

In [53]:
#Members who don't have a second U071 diagnosis
len(set(nc)-set(c))/len(df_m['clm_patient_id'].unique())

0.2387526615839

In [54]:
len(set(nc)-set(c))

3476

In [55]:
#Members that ONLY have a second U071 diagnosis
len(set(c) - set(nc))/len(df_m['clm_patient_id'].unique())

0.2132701421800948

In [52]:
len(set(c) - set(nc))

3105

In [56]:
df_m.to_csv('./data/confirmed_members/post_tableau.csv', index=False)

In [57]:
len(df_m['clm_patient_id'].unique())

14559

In [171]:
df_assoc = pd.get_dummies(df_m[df_m['cov_19']!='Confirmed'], columns=cols, sparse=True)

In [172]:
df_assoc.shape

(67972, 9550)

In [173]:
k = [x for x in df_assoc.columns if 'diag' in x]

In [174]:
len(k)

9543

In [175]:
df_assoc.head()

,cl_fst_serv_dt,cl_proc_cd,cov_19,invn_ctl_nbr,Unnamed: 0,clm_patient_id,days,clm_hc_diag_code1_51881,clm_hc_diag_code1_A0100,clm_hc_diag_code1_A042,...,clm_hc_diag_code5_Z95811,clm_hc_diag_code5_Z95828,clm_hc_diag_code5_Z96651,clm_hc_diag_code5_Z978,clm_hc_diag_code5_Z9861,clm_hc_diag_code5_Z98890,clm_hc_diag_code5_Z9911,clm_hc_diag_code5_Z992,clm_hc_diag_code5_Z9981,clm_hc_diag_code5_Z9989
1,2020-04-10,99214,Non-COVID,CA74024626,7754,89010861901F19720915,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,2020-04-11,99233,Probable,3437326744,443207,29585450118F19361108,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,2020-04-10,99233,Probable,3437326747,468027,29585450118F19361108,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,2020-04-15,99233,Probable,3438059017,521551,29585450118F19361108,10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2020-04-14,71045,Non-COVID,3438776091,742338,29585450118F19361108,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [176]:
#df = df_assoc[k][df_assoc[k].columns[df_assoc[k].sum()>10]]

In [177]:
df = df_assoc[k]

In [178]:
df.shape

(67972, 9543)

In [179]:
#k2 = [x for x in df.columns if 'R05' not in x]

In [180]:
#len(k2)

In [181]:
df = df[df.sum(axis=1) > 1]

In [182]:
df.shape

(40841, 9543)

In [183]:
df.shape[1] * .001

9.543000000000001

In [184]:
for i in df.columns:
    df[i] = df[i].astype('int8')

In [185]:
del df_assoc
gc.collect()

0

In [186]:
freq_items = apriori(df, min_support=0.001, use_colnames=True, verbose=1, low_memory=True)

Processing 20 combinations | Sampling itemset size 3


In [187]:
len(freq_items)

732

In [188]:
rules = association_rules(freq_items, metric="confidence", min_threshold=.6)

In [189]:
len(rules)

18

In [195]:
rules['antecedents'].iloc[15]

frozenset({'clm_hc_diag_code1_J1289', 'clm_hc_diag_code3_J9601'})

In [196]:
rules.sort_values(by=['support'], ascending=False).head(11)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(clm_hc_diag_code2_E870),(clm_hc_diag_code1_N179),0.005436,0.053206,0.003330,0.612613,11.513903,0.003041,2.444049
9,(clm_hc_diag_code2_Z992),(clm_hc_diag_code1_N186),0.004407,0.013810,0.002963,0.672222,48.677709,0.002902,3.008716
16,"(clm_hc_diag_code3_J9601, clm_hc_diag_code2_B9...",(clm_hc_diag_code1_J1289),0.003844,0.026420,0.002889,0.751592,28.448363,0.002788,3.919285
15,"(clm_hc_diag_code3_J9601, clm_hc_diag_code1_J1...",(clm_hc_diag_code2_B9729),0.003967,0.033398,0.002889,0.728395,21.809665,0.002757,3.558854
6,(clm_hc_diag_code2_E875),(clm_hc_diag_code1_N179),0.004481,0.053206,0.002791,0.622951,11.708207,0.002553,2.511061
8,(clm_hc_diag_code2_R34),(clm_hc_diag_code1_N179),0.002693,0.053206,0.002449,0.909091,17.086140,0.002305,10.414730
3,(clm_hc_diag_code2_J9602),(clm_hc_diag_code1_J9601),0.003208,0.089273,0.002228,0.694656,7.781258,0.001942,2.982631
10,(clm_hc_diag_code2_Z978),(clm_hc_diag_code1_R918),0.002938,0.035381,0.001861,0.633333,17.900323,0.001757,2.630779
7,(clm_hc_diag_code2_E8770),(clm_hc_diag_code1_N179),0.001689,0.053206,0.001322,0.782609,14.708938,0.001232,4.355251
0,(clm_hc_diag_code2_R6520),(clm_hc_diag_code1_A419),0.002032,0.015377,0.001249,0.614458,39.960306,0.001217,2.553867


In [197]:
df_g = pd.read_csv('data/COVID_vs_NON_regional/20200411.csv')

/usr/local/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [198]:
df_g.head()

,Unnamed: 0,cl_fst_serv_dt,invn_ctl_nbr,clm_hc_diag_code1,clm_hc_diag_code2,clm_hc_diag_code3,clm_hc_diag_code4,clm_hc_diag_code5,clm_hc_diag_code6,clm_hc_diag_code7,...,cl_proc_cd,cl_place_of_srcv,clm_patient_zip,clm_patient_state,clm_bill_prov_state,clm_bill_prov_zip,clm_patient_dob,clm_total_charge,cl_line_item_chrg_amt,cov_19
0,0,20200318,3437309047,I2510,I4901,NaN,NaN,NaN,NaN,NaN,...,93458,21,98024.0,WA,HI,968200060,19441115,766.00,766.00,Non-COVID
1,1,20200309,3437309109,S065X9A,S0181XA,NaN,NaN,NaN,NaN,NaN,...,99232,21,74604.0,OK,KS,672083052,19410612,450.00,150.00,Non-COVID
2,4,20200304,3437309119,R1032,J40,R319,E119,I509,I10,R5382,...,99214,11,75651.0,TX,TX,756052697,19550330,264.00,180.00,Non-COVID
3,8,20200327,3437309260,R000,NaN,NaN,NaN,NaN,NaN,NaN,...,93010,23,802033745.0,CO,TX,752677931,19801006,28.28,28.28,Non-COVID
4,9,20200312,3437309385,G43719,NaN,NaN,NaN,NaN,NaN,NaN,...,J0585,11,95070.0,CA,NV,891937297,19551105,3000.00,3000.00,Non-COVID
